# APLICACIÓN WEB DE TRANSCRIPCIÓN DE VOZ A TEXTO

![Logo de Python](./text2speech.jpg)

# Características Arquitectónicas Principales:
### 1. Soporte Dual de Modelos:
- Primario: Pipeline de Hugging Face Transformers
- Alternativo: Biblioteca Whisper de OpenAI
- Cambio automático si falla el método primario

### 2. Métodos Duales de Entrada:
- Grabación con Micrófono: Captura de audio en tiempo real
- Subida de ficheros: Archivos de audio pregrabados
- Sistema de prioridad: Las subidas anulan las grabaciones

### 3. Gestión de Archivos:
- Creación automática del directorio para guardar las grabaciones
- Crea ficheros únicos basados en marcas de tiempo
- Estandarización de audio (mono, 16kHz)
- Visualización clara de las rutas de los ficheros

### 4. Manejo de Errores:
- Informa al fallar la carga del modelo
- Validación de entrada
- Resolución de conflictos de puertos
- Mensajes de error amigables para el usuario

### 5. Experiencia de Usuario:
- Diseño limpio de dos columnas
- Separación clara entrada/salida
- Capacidad de compartir públicamente (Gradio UI)

### 6. Desarrollo
- Código ipynb (Jupyter)

In [260]:
# =============================================================
# APLICACIÓN WEB DE TRANSCRIPCIÓN DE VOZ A TEXTO
#==============================================================
# Tecnologías:
# - Gradio: interfaz web
# - Whisper: reconocimiento automático del habla (ASR)
# - PyTorch: detección de GPU
# - FFmpeg: normalización y guardado de audio
#
# Funcionalidades:
# - Grabar audio desde micrófono
# - Subir archivos de audio
# - Transcribir voz a texto
# - Guardar automáticamente el audio procesado
# - Interfaz centrada con columnas fijas y mismo tamaño
# =============================================================

# =================================
# 1. IMPORTACIÓN DE LIBRERÍAS
# =================================

import gradio as gr               # Gradio permite crear interfaces web interactivas de forma sencilla
from transformers import pipeline # pipeline simplifica el uso de modelos preentrenados de Hugging Face
import torch                      # Torch se usa aquí para comprobar si hay GPU disponible (CUDA)
import os                         # os permite interactuar con el sistema operativo (rutas, carpetas)
import ffmpeg                     # ffmpeg se usa para convertir y normalizar audio
import datetime                   # datetime se usa para generar timestamps únicos
import base64                     # para crear el logo en memoria
import socket                     # para verificar si hay un puerto URL libre
import json                       # para la cache a usar en los fichero que se cargan
import time                       # usar la hora del sistema

In [261]:
# =================================
# 2. INICIALIZACION DEL MODELO
# =================================

print("=" * 60)                                # Imprime una línea decorativa en consola
print("🎤 INICIALIZANDO TRANSCRIPTOR DE VOZ") # Mensaje informativo
print("=" * 60)

# -----------------------------
# SELECCIÓN DEL DISPOSITIVO
# -----------------------------
device = "cuda" if torch.cuda.is_available() else "cpu" # torch.cuda.is_available() devuelve True si hay GPU compatible
print(f"Dispositivo seleccionado: {device}")            # Se informa al usuario qué dispositivo se está usando

# -----------------------------
# CARGA DEL MODELO WHISPER
# -----------------------------
# Modelo PRINCIPAL para 99 idiomas
print("📥 Cargando modelo Whisper (openai/whisper-small)")
print("✅ Idiomas soportados: inglés, español, portugués, chino, italiano + 94 más")

try:
    # Intentar cargar el modelo Whisper usando el pipeline de Hugging Face (hf) transformers
    # "automatic-speech-recognition" es el tipo de tarea para conversión de voz a texto
    # "openai/whisper-small" es una variante específica del modelo Whisper PRE-ENTRENADO
    # El parámetro device asegura que el modelo se cargue en GPU si está disponible
    
    # Carga el modelo preentrenado usando HF
    pipe = pipeline(
        "automatic-speech-recognition",
        model="openai/whisper-small",
        device=device)

    # Flag para rastrear qué implementación del modelo se está usando
    USE_WHISPER = False # Usando el pipeline de transformers, no la biblioteca OpenAI Whisper
    
    print("✔ Whisper cargado mediante Transformers ¡EXITOSAMENTE!") # Mensaje de éxito para la pipeline de transformers

except Exception:
    # Plan B: Si falla el pipeline de transformers (dependencias, versiones, etc.), usar la biblioteca OpenAI Whisper directamente
    # se usa la librería oficial de OpenAI Whisper
    import whisper

    # Cargar el modelo Whisper "base" (variante más pequeña que equilibra velocidad/precisión)
    whisper_model = whisper.load_model("base")
    USE_WHISPER = True # Flag que indica que estamos usando la biblioteca OpenAI Whisper

    print("✔ Whisper cargado mediante OpenAI (fallback)") # Informar al usuario sobre el método alternativo


🎤 INICIALIZANDO TRANSCRIPTOR DE VOZ
Dispositivo seleccionado: cuda
📥 Cargando modelo Whisper (openai/whisper-small)
✅ Idiomas soportados: inglés, español, portugués, chino, italiano + 94 más


Device set to use cuda


✔ Whisper cargado mediante Transformers ¡EXITOSAMENTE!


In [262]:
# =================================
# 3. DIRECTORIO DE GRABACIONES
# =================================

# Nombre de la carpeta donde se guardarán los audios
RECORDINGS_DIR = "_recordings"

# os.makedirs crea la carpeta si no existe
# exist_ok=True evita error si el directorio ya existe
os.makedirs(RECORDINGS_DIR, exist_ok=True)

# Mostrar ruta absoluta para referencia del usuario (ruta completa desde la raíz)
print(f"Carpeta de grabaciones: {os.path.abspath(RECORDINGS_DIR)}")

Carpeta de grabaciones: c:\Users\castr\Desktop\syncSchool\OBS\10_Speech-Next_Analytics\_recordings


In [263]:
# =================================
# 4. FUNCIÓN DE TRANSCRIPCIÓN
# =================================

def transcribe_audio_ori(audio_path, uploaded_file=None):
    """
    Transcribe audio desde grabación de micrófono (graba la voz en un fichero antes de trancribir) o archivo cargado a memoria.
    
    Parámetros:
    -----------
    audio_path : str
        Ruta del archivo de audio desde la grabación usando el micrófono
        
    uploaded_file : str, opcional
        Ruta del archivo de audio subido (tiene prioridad sobre audio_path)
    
    Retorna:
    --------
    tuple : (texto_transcrito, información_del_archivo)
        Transcripción de texto y detalles sobre el archivo guardado
    """
    
    # Sistema de prioridad: archivo subido anula la grabación del micrófono
    # Si el usuario sube un archivo, se usa en lugar de la grabación
    if uploaded_file is not None:
        audio_path = uploaded_file

    # Validación básica: si no hay audio, se devuelve mensaje
    if not audio_path:
        return "No hay audio para transcribir", "No hay archivo" # Mensaje de error si no hay fichero de audio

    try:
        # Se genera un timestamp para evitar sobrescribir archivos
        # Genera nombre de fichero único mediante fecha_hora
        # Formato: AñoMesDía_HoraMinutoSegundo (ej: 20240115_143025)
        timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")

        # Nombre del fichero
        if uploaded_file:
            # Para ficheros subidos: preservar nombre original con marca de tiempo
            original_name = os.path.basename(uploaded_file)        # Obtener solo el nombre del fichero sin ruta
            name_without_ext = os.path.splitext(original_name)[0]  # Remover extensión del fichero
            filename = f"{name_without_ext}_{timestamp}.wav"       # Agregar marca de tiempo y extensión .wav
        else:
            # Para grabaciones de micrófono: nombre genérico con marca de tiempo
            filename = f"recording_{timestamp}.wav"

        # Ruta completa donde se guarda el audio
        save_path = os.path.join(RECORDINGS_DIR, filename)

        # -------------------------
        # LÓGICA DE TRANSCRIPCIÓN
        # -------------------------
        # Elegir método de transcripción basado en qué modelo se cargó exitosamente
        if USE_WHISPER:
            # Usar biblioteca OpenAI Whisper
            result = whisper_model.transcribe(audio_path) # Devuelve un diccionario con clave "text"
            text = result.get("text", "")                 # Extrae el texto de transcripción
        else:
            # El pipeline Transformers de HF devuelve un diccionario
            result = pipe(audio_path)     # Devuelve diccionario con clave "text"
            text = result.get("text", "") # Extrae el texto de transcripción

        # ====================================================================
        # PROCESAMIENTO Y GUARDADO DE AUDIO
        # ====================================================================
        
        # Convertir y guardar audio a formato estandarizado usando ffmpeg
        # ffmpeg.input() - Especifica fichero de audio de entrada
        # .output() - Define parámetros de salida:
        #   save_path - Ruta del fichero de destino
        #   ac=1 - Convertir a mono (1 canal de audio)
        #   ar=16000 - Remuestrear a 16kHz (óptimo para reconocimiento de voz)
        # .run(quiet=True) - Ejecutar conversión silenciosamente (sin salida en consola)
        
        # -------------------------
        # NORMALIZACIÓN DEL AUDIO
        # -------------------------

        # ffmpeg.input carga el audio original
        # output:
        # - ac=1 → mono
        # - ar=16000 → 16 kHz
        ffmpeg.input(audio_path).output(save_path,
                                        ac=1,
                                        ar=16000).run(quiet=True)

        # Ruta absoluta (ruta completa desde la raíz)
        full_path = os.path.abspath(save_path)
        
        # Crear mensaje de información del fichero amigable para el usuario
        file_info = f"Guardado en: {full_path}"
        
        # Retornar resultados: texto de transcripción (sin espacios en blanco) e información del fichero
        return text.strip(), file_info

    except Exception as e:
        # Captura cualquier error y lo devuelve a la UI
        # Manejo de errores: retornar mensaje de error si algo falla
        return f"Error durante la transcripción: {str(e)}", ""

## CSS para:
✅ Efectos visuales (sombras, gradientes, transiciones)

✅ Responsive complejo (media queries)

✅ Control pixel-perfect

✅ Temas y estilos personalizados

✅ Animaciones y micro-interacciones

In [264]:
# =====================================
# 5. Definir parámetros CSS
# =====================================

CSS = """
/* Variables globales */
:root{--col-width:600px;
      --gap:24px;
      --radius:8px;
      --shadow:0 6px 18px rgba(0,0,0,.08)}

/* Centrado general */
.gradio-root{display:flex;
             justify-content:center}

/* Contenedor común de filas */
.center-wrapper{display:flex;
                gap:var(--gap);
                justify-content:center;
                align-items:flex-start;
                max-width:calc(var(--col-width)*2 + var(--gap));
                width:100%;
                margin:16px auto}

/* Columnas estándar */
.equal-column{width:var(--col-width)!important;
              background:#fff;
              border-radius:var(--radius);
              box-shadow:var(--shadow);
              padding:12px}

/* Columna que ocupa ambas */
.span-two-columns{width:calc(var(--col-width)*2 + var(--gap))!important}

/* Botón ancho completo */
#transcribe-btn{width:100%!important}

/* Responsive */
@media (max-width:1240px){.center-wrapper{flex-direction:column},
                          .equal-column,
                          .span-two-columns{width:100%!important}}
"""

# Configurar columnas Gradio

In [265]:
# Define el comportamiento de las columnas en Gradio
LAYOUT = {"col_l": {"scale": 1, "min_width": 600},
          "col_r": {"scale": 1, "min_width": 600}}

# Cache to use the uploaded files once

In [266]:
class AudioCache:
    """Manage audio file cache"""
    
    def __init__(self, cache_dir=RECORDINGS_DIR):
        self.cache_dir = cache_dir
        self.cache_file = os.path.join(cache_dir, "cache_index.json")
        self.cache = self.load_cache()
        
    def load_cache(self):
        """Load cache index from file"""
        if os.path.exists(self.cache_file):
            with open(self.cache_file, 'r') as f:
                return json.load(f)
        return {}
    
    def save_cache(self):
        """Save cache index to file"""
        with open(self.cache_file, 'w') as f:
            json.dump(self.cache, f, indent=2)
    
    def get_cached_file(self, original_path):
        """Get cached processed file if exists"""
        file_hash = self._get_file_signature(original_path)
        
        if file_hash in self.cache:
            cached_path = self.cache[file_hash]['processed_path']
            # Check if file still exists
            if os.path.exists(cached_path):
                return cached_path
        
        return None
    
    def add_to_cache(self, original_path, processed_path):
        """Add file to cache"""
        file_hash = self._get_file_signature(original_path)
        self.cache[file_hash] = {
            'original': original_path,
            'processed_path': processed_path,
            'timestamp': time.time(),
            'size': os.path.getsize(processed_path)
        }
        self.save_cache()
    
    def _get_file_signature(self, file_path):
        """Create unique signature for file"""
        stat = os.stat(file_path)
        # Combine filename, size, and modification time
        return f"{os.path.basename(file_path)}_{stat.st_size}_{int(stat.st_mtime)}"

In [267]:
# runs the cache
audio_cache = AudioCache()

In [268]:
def transcribe_audio(audio_path, uploaded_file=None):
    """Smart file management: Don't create duplicates for uploaded files"""
    
    if uploaded_file is not None:
        audio_path = uploaded_file

    if not audio_path:
        return "No hay audio para transcribir", "No hay archivo"

    try:
        # ==========================================
        # ALWAYS TRANSCRIBE FROM ORIGINAL FIRST
        # ==========================================
        print(f"📝 Transcribiendo: {os.path.basename(audio_path)}")
        
        if USE_WHISPER:
            result = whisper_model.transcribe(audio_path)
            text = result.get("text", "")
        else:
            result = pipe(audio_path)
            text = result.get("text", "")
        
        # ==========================================
        # SMART FILE PROCESSING
        # ==========================================
        if uploaded_file:
            original_name = os.path.basename(audio_path)
            name_without_ext = os.path.splitext(original_name)[0]
            original_ext = os.path.splitext(original_name)[1].lower()
            
            # Determine if we already have this file processed
            save_path = None
            file_info = ""
            
            # Check recordings directory for matching files
            for file in os.listdir(RECORDINGS_DIR):
                file_lower = file.lower()
                
                # Case 1: Exact same filename already exists (rare but possible)
                if file == original_name:
                    save_path = os.path.join(RECORDINGS_DIR, file)
                    file_info = f"Archivo ya existe en grabaciones: {file}"
                    print(f"✅ Archivo ya existe: {file}")
                    break
                
                # Case 2: Processed WAV version exists (e.g., audio.mp3 -> audio.wav)
                elif (file_lower.startswith(name_without_ext.lower()) and 
                      file_lower.endswith('.wav') and
                      not file_lower.startswith('recording_')):
                    save_path = os.path.join(RECORDINGS_DIR, file)
                    file_info = f"Usando versión procesada existente: {file}"
                    print(f"✅ Versión procesada existe: {file}")
                    break
            
            if not save_path:
                # No existing file found, create processed version
                if original_ext == '.wav':
                    # If uploading a WAV file, check if it's already in the right format
                    # before deciding to copy or process it
                    try:
                        import soundfile as sf
                        info = sf.info(audio_path)
                        
                        # Check if it's already mono, 16kHz
                        if info.channels == 1 and info.samplerate == 16000:
                            # Already in correct format, just copy it
                            filename = original_name
                            save_path = os.path.join(RECORDINGS_DIR, filename)
                            
                            # Copy file instead of reprocessing
                            import shutil
                            shutil.copy2(audio_path, save_path)
                            
                            file_info = f"Archivo WAV ya en formato correcto, copiado: {filename}"
                            print(f"📋 WAV ya en formato correcto, copiado: {filename}")
                        else:
                            # Needs processing
                            filename = f"{name_without_ext}.wav"
                            save_path = os.path.join(RECORDINGS_DIR, filename)
                            
                            ffmpeg.input(audio_path).output(save_path,
                                                          ac=1,
                                                          ar=16000).run(quiet=True)
                            
                            file_info = f"WAV procesado a formato estándar: {filename}"
                            print(f"🔧 WAV procesado a formato estándar: {filename}")
                            
                    except Exception as e:
                        # Fallback: just process it
                        filename = f"{name_without_ext}.wav"
                        save_path = os.path.join(RECORDINGS_DIR, filename)
                        
                        ffmpeg.input(audio_path).output(save_path,
                                                      ac=1,
                                                      ar=16000).run(quiet=True)
                        
                        file_info = f"Procesado: {filename}"
                        print(f"💾 Procesado (fallback): {filename}")
                        
                else:
                    # For non-WAV files (mp3, m4a, etc.)
                    filename = f"{name_without_ext}.wav"
                    save_path = os.path.join(RECORDINGS_DIR, filename)
                    
                    ffmpeg.input(audio_path).output(save_path,
                                                  ac=1,
                                                  ar=16000).run(quiet=True)
                    
                    file_info = f"Convertido a WAV: {filename}"
                    print(f"🔄 Convertido a WAV: {filename}")
                    
        else:
            # For recordings: Always create new file with timestamp
            timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
            filename = f"recording_{timestamp}.wav"
            save_path = os.path.join(RECORDINGS_DIR, filename)
            
            ffmpeg.input(audio_path).output(save_path,
                                          ac=1,
                                          ar=16000).run(quiet=True)
            
            file_info = f"Grabación nueva: {filename}"
            print(f"🎤 Nueva grabación: {filename}")
        
        full_path = os.path.abspath(save_path)
        file_info += f"\n📁 Ubicación: {full_path}"
        
        return text.strip(), file_info

    except Exception as e:
        print(f"❌ Error: {str(e)}")
        return f"Error durante la transcripción: {str(e)}", ""

In [ ]:
# ============================================================================
# 6 CONFIGURACIÓN DE LA INTERFAZ DE USUARIO GRADIO
# ============================================================================
# Crear interfaz de Bloques de Gradio (más flexible que la Interface simple)
demo = gr.Blocks(css=CSS)

# Definir diseño y componentes de la UI
with demo:

    # ENCABEZADO
    with gr.Row(elem_classes="center-wrapper"):
        with gr.Column(elem_classes=["equal-column"]):
            # permite descargar, ampliar y compartir la imagen
            # gr.Image(value='./text2speech.jpg', interactive=False, type="filepath")
            
            # solo para ficheros locales, no vale para internet
            # gr.Markdown("""
            #     <div style="pointer-events: none; user-select: none; width:600px; margin:auto;">
            #         <img src="./text2speech.jpg" style="width:100%; height:auto; user-drag:none; pointer-events:none;">
            #     </div>
            #     """)

            # Abrir el logo y codificarlo a base64
            with open("./text2speech.jpg", "rb") as f:
                data = f.read()
            logo_b64 = base64.b64encode(data).decode()

            # Insertar logo en Markdown/HTML
            gr.Markdown(f"""
                        <div style="pointer-events: none; user-select: none; width:600px; margin:auto;">
                            <img src="data:image/jpeg;base64,{logo_b64}" style="width:100%; height:auto; user-drag:none; pointer-events:none;">
                        </div>
                        """)
            

        with gr.Column(elem_classes=["equal-column"]):
            # Título y descripción
            gr.Markdown('# Actividad "Speech and Text Analytics"')
            gr.Markdown("## Ejercicio #1\n")
            
            gr.Markdown("## 🎤 Voz a Texto")
            gr.Markdown(f"**📁 Las grabaciones se guardadan en:**")
            # full_path = os.path.join(os.path.dirname(os.path.abspath(RECORDINGS_DIR)), RECORDINGS_DIR)
            gr.Textbox(value = os.path.abspath(RECORDINGS_DIR),
                       label = "📁 Ruta de grabaciones",
                       interactive = False)

    # =================================================================================
    # SECCIÓN DE ENTRADA - Diseño de dos columnas para diferentes métodos de entrada
    # =================================================================================
    with gr.Row(elem_classes="center-wrapper"): # Contenedor horizontal en fila
        with gr.Column(elem_classes=["equal-column"]): # Columna de la izquierda para grabación con micrófono
            gr.Markdown("### Opción 1: Grabar con Micrófono")

            # Componente de grabador de audio
            # sources=["microphone"] - Habilitar solo grabación con micrófono
            # type="filepath"        - El componente retorna la ruta al archivo de audio temporal
            # label="Grabar Audio"   - Etiqueta de visualización para el componente
            audio_input = gr.Audio(sources = ["microphone"],
                                   type = "filepath",
                                   label = 'Grabar Audio')

        with gr.Column(elem_classes=["equal-column"]): # Columna de la derecha para subir ficheros
            gr.Markdown("### Opción 2: Subir fichero de Audio")
            
            # Componente de subida de ficheros
            # label="Subir fichero de Audio" - Etiqueta de visualización
            # file_types - Lista de formatos de audio soportados
            # type="filepath" - Retorna ruta al archivo subido
            
            file_input = gr.File(label = "Fichero de Audio (.wav .mp3 .m4a .flac .ogg .acc))",
                                 file_types = [".wav", ".mp3", ".m4a", ".flac", ".ogg", ".aac"],
                                 type = "filepath")

    # ========================================================================
    # BOTÓN DE ACCIÓN
    # ========================================================================
    
    # Botón de acción principal que activa la transcripción
    # variant="primary" - Estilo para que destaque como acción principal
    with gr.Row(elem_classes="center-wrapper"):
        with gr.Column(elem_classes=["span-two-columns"]):
            transcribe_btn = gr.Button("Transcribir",
                                       variant = "primary",
                                       elem_id = "transcribe-btn")

    # ========================================================================
    # SECCIÓN DE SALIDA - Diseño de dos columnas para resultados
    # ========================================================================
    with gr.Row(elem_classes="center-wrapper"):
        output_text = gr.Textbox(label="Transcripción", lines=5)         # Columna izquierda: Texto de transcripción
        file_info = gr.Textbox(label="Información del archivo", lines=5) # Columna derecha..: Información del fichero

    # ========================================================================
    # CONEXIÓN BOTÓN → FUNCIÓN
    # MANEJO DE EVENTOS - Conectar clic del botón a la función
    # ========================================================================
    
    # Cuando se hace clic en transcribe_btn:
    # 1. Llama la función transcribe_audio()
    # 2. Pasa el audio_input y file_input como argumentos
    # 3. Actualiza el output_text y file_info con los valores de retorno de la función
    transcribe_btn.click(
        transcribe_audio,
        inputs=[audio_input, file_input],
        outputs=[output_text, file_info])
    
    # ============================================================================
    # Team Credits
    # ============================================================================
    gr.Markdown("---")
    gr.Markdown("""
        <div style='text-align: center; padding: 20px;'>
            <h3>👨‍💻 Development Team</h3>
            <p><strong>🤖 Lead Developer:</strong> Riccardo</p>
            <p><strong>📈 Data Scientist:</strong> Carlos and Juan</p>
            <p><strong>🔧 ML Engineer:</strong> Nicolas</p>
            <br>
            <p>📬 Contact: obstfmgrupo1@gmail.com | 🌐 Website: <a href='https://i.ibb.co/WNRC7fPn/text2speech.jpg'>Speech 2 Text</a></p>
            <p>💻 <a href='https://github.com/whaleskin/Speech2Text.git'>GitHub Repository</a></p>
            <p style='color: gray; font-size: 12px;'>© 2026 OBS Grupo 1 - TRANSCRIPCIÓN DE VOZ A TEXTO. All rights reserved.</p>
        </div>
        """)

C:\Users\castr\AppData\Local\Temp\ipykernel_21340\694947360.py:5: UserWarning: The parameters have been moved from the Blocks constructor to the launch() method in Gradio 6.0: css. Please pass these parameters to launch() instead.
  demo = gr.Blocks(css=CSS)


In [270]:
# ============================================
# Verifica si hay un puerto disponible
# ============================================

def find_available_port(start_port: int = 7860, max_attempts: int = 100):
    """Find an available port starting from start_port."""
    for port in range(start_port, start_port + max_attempts):
        # Verificar rápidamente si el puerto está disponible
        with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as sock:
            sock.settimeout(1)
            if sock.connect_ex(('127.0.0.1', port)) == 0:
                    print(f"❌ Puerto {port} está ocupado, probando siguiente...")
                    continue
            else:
                return port

# Asigna el puerto a una variable para ser usado por Gradio
available_port = find_available_port(7860)
print(f"✅ Available port: {available_port}")

❌ Puerto 7860 está ocupado, probando siguiente...
❌ Puerto 7861 está ocupado, probando siguiente...
❌ Puerto 7862 está ocupado, probando siguiente...
✅ Available port: 7863


In [271]:
# ============================================================================
# 7 LANZAMIENTO DE LA APLICACIÓN
# ============================================================================

if __name__ == "__main__":
    """
    Bloque de ejecución principal - solo se ejecuta cuando el script se ejecuta directamente
    (no cuando se importa como un módulo)
    """
    
    # Mostrar información del sistema para el usuario
    print(f"\nDirectorio actual: {os.getcwd()}")  # Mostrar dónde se guardarán los archivos
    print("Carpeta de grabaciones: ./recordings/")  # Ruta relativa a las grabaciones
    
    print("\nIniciando aplicación...")  # Notificación de lanzamiento
    print(f"URL: http://localhost:{available_port}")  # URL de acceso local
    
    # Instrucciones para el usuario para apagado adecuado
    print("\n⚠️  IMPORTANTE: Para detener la aplicación correctamente:")
    print("1. Haz clic en el botón DETENER en VS Code")
    print("2. O Presiona Ctrl+C en la terminal")
    print("3. Espera 5 segundos antes de reiniciar")
    print("\nLa aplicación permanecerá abierta hasta que la detengas.")
    
    print("\n🚀 INICIANDO APLICACIÓN DE TRANSCRIPCIÓN")
    print("="*50)
    print(f"🌐 URL: http://localhost:{available_port}")
    print("🎤 Modelo: Whisper-small")
    print(f"💻 Dispositivo: {device}")
    print("="*50)
    
    print("\n📋 INSTRUCCIONES:")
    print("1. Graba audio con el micrófono")
    print("2. O sube un archivo de audio")
    print("3. Haz clic en 'Transcribir'")
    print("\n🛑 PARA DETENER: Usa el botón DETENER en VS Code")
    
    launched_successfully = False
    try:
        # Lanzar la aplicación web de Gradio
        # server_name="127.0.0.1"  - Solo localhost (no accesible desde la red)
        # server_port=7860         - Puerto por defecto de Gradio
        # show_error=True          - Mostrar errores en la UI
        # quiet=False              - Mostrar mensajes de inicio de Gradio
        # share=True               - Crear URL pública temporal (accesible desde internet)
        # debug=True               - Habilitar modo depuración
        # prevent_thread_lock=True - Permite que VS Code detenga la aplicación correctamente
        
        demo.launch(server_name = "127.0.0.1", 
                    server_port = available_port, #show_error = True, quiet = False, prevent_thread_lock = True)  # Importante: permite que VS Code lo detenga
                    share = True,
                    debug = True)
        launched_successfully = True
    except KeyboardInterrupt:
        # Manejar interrupción manual (Ctrl+C)
        print("\n\nAplicación detenida por el usuario.")
    except Exception as e:
        # Manejar otras excepciones
        print(f"\nError: {e}")
        
        # Manejo especial para cuando el puerto 7860 está en uso
        if "7860" in str(e):
            print(f"¡El puerto {available_port} está ocupado!")
            print("Se han probando hasta 100 puertos...")
            print("Solución: Cierra VS Code y reinicia el programa.")
    finally:
        # ESTO SIEMPRE SE EJECUTA, INCLUYDO CUANDO VS CODE DETIENE
        if launched_successfully:
            print("\n" + "✅" * 20)
            print("✅ APLICACIÓN FINALIZADA CORRECTAMENTE")
            print("✅" * 20)
            print("\n📊 RESUMEN:")
            print(f"• Archivos procesados en: {os.path.abspath(RECORDINGS_DIR)}")
            print("• Puedes volver a ejecutar el programa cuando quieras")
            print("• Los archivos se mantienen para futuras transcripciones")
            print("\n👋 ¡Gracias por usar el transcriptor de voz!")



Directorio actual: c:\Users\castr\Desktop\syncSchool\OBS\10_Speech-Next_Analytics
Carpeta de grabaciones: ./recordings/

Iniciando aplicación...
URL: http://localhost:7863

⚠️  IMPORTANTE: Para detener la aplicación correctamente:
1. Haz clic en el botón DETENER en VS Code
2. O Presiona Ctrl+C en la terminal
3. Espera 5 segundos antes de reiniciar

La aplicación permanecerá abierta hasta que la detengas.

🚀 INICIANDO APLICACIÓN DE TRANSCRIPCIÓN
🌐 URL: http://localhost:7863
🎤 Modelo: Whisper-small
💻 Dispositivo: cuda

📋 INSTRUCCIONES:
1. Graba audio con el micrófono
2. O sube un archivo de audio
3. Haz clic en 'Transcribir'

🛑 PARA DETENER: Usa el botón DETENER en VS Code
* Running on local URL:  http://127.0.0.1:7863
* Running on public URL: https://a228b9657bfe73ef9e.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)

Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7863 <> https://a228b9657bfe73ef9e.gradio.live

✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅
✅ APLICACIÓN FINALIZADA CORRECTAMENTE
✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅

📊 RESUMEN:
• Archivos procesados en: c:\Users\castr\Desktop\syncSchool\OBS\10_Speech-Next_Analytics\_recordings
• Puedes volver a ejecutar el programa cuando quieras
• Los archivos se mantienen para futuras transcripciones

👋 ¡Gracias por usar el transcriptor de voz!


### eof()